In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
L=5

dim = np.zeros((L+1,))

dim[0] = 5
dim[1] = 5
dim[2] = 5
dim[3] = 5
dim[4] = 5
dim[5] = 5

dim= dim.astype(int)

npar = 0
cpar = np.zeros((L+1,))
for iL in range(L):
    npar = npar + dim[iL] * dim[iL+1]
    cpar[iL+1] = npar
npar = npar.astype(int)
cpar = cpar.astype(int)

print(npar)
    
num_train_data = 500
num_test_data = 500

#num_train_data = 150
#num_test_data = 150

noise = 0

125


### Create Neural Network

In [3]:

import torch.nn as nn
import torch.nn.functional as F


# No bias with ReLU implies homogeneous function of the input 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        layers = [nn.Linear(dim[0],dim[1],bias=False)]
        for iL in np.arange(1,L):
            layers.append(nn.ReLU())
            layers.append(nn.Linear(dim[iL],dim[iL+1],bias=False))        
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)



# Define a custom initialization function using Gaussian distribution
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        # Gaussian initialization 
        y = m.in_features
        nn.init.normal_(m.weight, mean=0.0, std=1/np.sqrt(y)) 
        #nn.init.normal_(m.bias, mean=0.0, std=1.0)  


#net_teacher = Net()
#net_teacher.apply(initialize_weights)

net = Net()
net.apply(initialize_weights)

net0 = Net()
net0.load_state_dict(net.state_dict())

num_parameters = sum(param.numel() for param in net.parameters())
print(num_parameters)


125


In [ ]:
# print list of all parameter tensors
print(list(net.parameters()))

### Input data

In [4]:

#Sigma, R = np.linalg.qr(np.random.normal(np.zeros((dim[0],dim[0])),1))
#Sigma = Sigma @ np.diag(np.logspace(0,-3,dim[0])) @ Sigma.T
#inputs_train = torch.from_numpy(np.random.multivariate_normal(np.zeros((dim[0],)),Sigma,size=(num_train_data,))).to(torch.float)
#inputs_test = torch.from_numpy(np.random.multivariate_normal(np.zeros((dim[0],)),Sigma,size=(num_test_data,))).to(torch.float)

inputs_train =  torch.rand((num_train_data,1))# @ torch.ones((1, dim[0]))
inputs_test = torch.rand((num_test_data,1))# @ torch.ones((1, dim[0]))





### Compute Jacobian

In [ ]:

outputs_train = net(inputs_train)
outputs_vec = outputs_train.view(-1)

external_grad = torch.eye(num_train_data)

Jac = torch.zeros(num_train_data,num_parameters)
for iout in range(num_train_data):
    
    net.zero_grad()

    outputs_vec.backward(external_grad[iout],retain_graph=True)
    
    grad_vec = []
    for param in net.parameters():
        #grad_vec.append(param.grad.view(-1))
        grad_vec.append(param.grad.T.reshape(-1,1))
    grad_vec = torch.cat(grad_vec)
    
    Jac[iout,:] = grad_vec.T

if num_train_data >= num_parameters:
    U, S, V = torch.svd(Jac, some=False, compute_uv=True)
    evl = torch.zeros(num_train_data,1)
    evl[:num_parameters,:] = S.unsqueeze(1)

if num_train_data < num_parameters:
    U, S, V = torch.svd(Jac, some=True, compute_uv=True)
    evl = S**2
    evl = evl.unsqueeze(1)

    

In [ ]:

plt.plot(S,marker='o')
plt.yscale('log')
plt.show()

plt.imshow(Jac.T@Jac)
plt.colorbar()
plt.clim((-10,10))
plt.show()
